In [1]:
import pandas as pd
import rasterio
import geopandas as gpd
import os
import rasterstats
import matplotlib.pyplot as plt
import seaborn as sns
import natsort
from natsort import natsorted
sns.set(rc={'figure.figsize':(15, 10)})

In [3]:
snakemake   

In [2]:
#get electricity consumption for lau units
elec_lau1 = pd.read_excel(
    'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/853754/fxlsx',
    sheet_name='2014r', header=[0, 1, 2])

elec_lau1 = elec_lau1.set_index(
    ('Sub-national electricity sales and numbers of customers, 2014 (1)(4)', 'Unnamed: 2_level_1', 'LA Code')
)[('Sales (GWh)', 'All')].dropna()
elec_lau1.index = elec_lau1.index.rename('LA Code')

# Shapefile of the LAU1 units
uk_lau1_units = gpd.read_file('https://opendata.arcgis.com/datasets/69cd46d7d2664e02b30c2f8dcc2bfaf7_0.geojson')
uk_lau1_units = uk_lau1_units.set_index('LAD19CD')[['LAD19NM', 'geometry']]
uk_lau1_units.index = uk_lau1_units.index.rename('LA Code')

# join shapefile and electricity demand
elec_lau1=gpd.GeoDataFrame(elec_lau1.join(uk_lau1_units).dropna(), geometry='geometry')

#import grided population data
with rasterio.open(r'C:\Users\silas\kDrive\Projects\BA\python\geodata\population-uk.tif') as pop_grid:
        array = pop_grid.read(1)
        crs = pop_grid.crs
        affine = pop_grid.transform
        
#importing Calliope model zones 
calliope_zones=gpd.read_file(r'C:\Users\silas\kDrive\Projects\BA\python\geodata\calliope_zones\zones.shp')


In [3]:
#overlay
lau_zones = gpd.overlay(calliope_zones.to_crs(crs), elec_lau1.to_crs(crs), how='intersection', keep_geom_type=False)

#calculate population per snippet
pop = rasterstats.zonal_stats(lau_zones.to_crs(crs), array, affine=affine, stats='sum', nodata=0)
lau_zones['population'] = [i['sum'] for i in pop]

# weighted average according to population
lau_zones = lau_zones.set_index(['Name_1', 'LAD19NM'], drop=False)
weights = lau_zones.population.div(lau_zones.sum(level='LAD19NM').population)
lau_zones['Total consumption']=lau_zones['Total consumption']*weights
elec_zones=lau_zones.groupby(level='Name_1').sum()
elec_zones=elec_zones['Total consumption']

In [5]:
#import grided population data
with rasterio.open(r'C:\Users\silas\kDrive\Projects\BA\python\geodata\population-uk.tif') as pop_grid:
        array = pop_grid.read(1)
        crs = pop_grid.crs
        affine = pop_grid.transform

In [4]:
#get load profile 
load_profile_all= pd.read_csv('load_profile_2014.csv', header=0, index_col=0)
load_profile_all.index=pd.to_datetime(load_profile_all.index).tz_localize(None)
load_profile=load_profile_all.loc['2014','GB_load_entsoe_power_statistics']

#normalise based on anual load
load_profile=load_profile/load_profile.sum()


In [6]:
#create demand dataframe
elec_demand=pd.concat([load_profile] * 20, axis=1)
elec_demand.columns=elec_zones.index
elec_demand=elec_demand.mul(elec_zones)
elec_demand=elec_demand*-1
elec_demand['EAST_ANGLIA']=0
elec_demand['HORNSEA']=0
elec_demand.index=pd.to_datetime(elec_demand.index)

#export to csv
elec_demand.to_csv('electricity_demand_2014.csv')

In [8]:
#prepare dataframe for plotting
calliope_zones.set_index('Name_1', inplace=True)
elec_zones=calliope_zones.join(elec_zones)
elec_zones.drop(['DOGGER_BANK', 'HORNSEA', 'EAST_ANGLIA'], axis=0, inplace=True)

elec_lau1.to_file(r'C:\Users\silas\kDrive\Projects\BA\python\geodata\elec_lau1.shp')
elec_zones.to_file(r'C:\Users\silas\kDrive\Projects\BA\python\geodata\elec_zones.shp')

In [9]:
elec_demand.drop(['EAST_ANGLIA', 'HORNSEA'], axis=1, inplace=True)

In [30]:
#calculate average hourly demand during jan, feb, november and december

elec_winter=elec_demand[((elec_demand.index >= '2014-01-01') & (elec_demand.index <= '2014-02-28')) 
    | ((elec_demand.index >= '2014-11-01') & (elec_demand.index < '2014-12-31'))]*-1

#sort zones naturally
elec_winter=elec_winter.T
elec_winter=elec_winter.reindex(index=natsorted(elec_winter.index))
elec_winter=elec_winter.T

#plot

ax=sns.boxplot(data=elec_winter, color='#4daf4a')
ax.set_title('Hourly electrictity demand in Britain during winter', fontsize=25)
ax.set_ylabel('GWh')
ax.set_xlabel('Calliope Zone')
ax.tick_params(axis='both', which='major', labelsize=15)
themes.theme_fivethirtyeight()

plt.savefig(r'C:\Users\silas\kDrive\Projects\BA\writing\plots\elec_winter_hourly.png', transparent=True)